# Prueba de las funciones


Este notebook se enfoca en desarrollar y probar funciones para los endpoints de la API, utilizando datasets específicos.


## Libreria utilizadas


En esta sección, importamos todas las bibliotecas y/o modulos necesarios para nuestro proceso de feature engineering y establecemos configuraciones globales de ser requerido.


In [2]:
import sys
import pandas as pd
import numpy as np


## Extracción


En esta sección, extraemos los datos de los archivos `df_endpoints1_2` y `df_endpoints3_4_5` en formato parquet que serán utilizados como si fuesen fuentes de base de datos para los endpoints.


### Extracción de los datos


Creamos una función que lee cada archivo desde su directorio y lo carga a un DataFrame de `pandas`.


In [3]:
# Cargamos los archivos parquet
def read_parquet_files(parquet_files):
    dataframes = {}
    for name in parquet_files:
        dataframes[name] = pd.read_parquet(
            f'../dataset/{name}.parquet', engine='pyarrow')
    return dataframes


parquet_files = ['df_endpoints1_2', 'df_endpoints3_4_5', 'df_similitud_usuarios', 'df_similitud_items', 'matrix']
dataframes = read_parquet_files(parquet_files)

# Convertimos a df los archivos parquet usando un iterador sobre la variable parquet_files y dataframes.
df_endpoints1_2 = next(dataframes[name] for name in parquet_files if name == 'df_endpoints1_2')
df_endpoints3_4_5 = next(dataframes[name] for name in parquet_files if name == 'df_endpoints3_4_5')
df_similitud_usuarios = next(dataframes[name] for name in parquet_files if name == 'df_similitud_usuarios')
df_similitud_items = next(dataframes[name] for name in parquet_files if name == 'df_similitud_items')
df_matrix = next(dataframes[name] for name in parquet_files if name == 'matrix')




## Endpoints


### Endpoint 1


def **PlayTimeGenre( _`genero` : str_ )**:
Retorna `año` con mas horas jugadas para el género dado.
Ejemplo de retorno:

```js
{
   "Año de lanzamiento con más horas jugadas para Género X": 2013
}
```


In [12]:
def PlayTimeGenre(genre: str):
    """
    Devuelve el año con más horas jugadas para un género dado.

    Args:
        genre: El género del juego.

    Returns:
        El año de lanzamiento con más horas jugadas para un género dado.
    Ejemplo de retorno:
        {
            "Año de lanzamiento con más horas jugadas para Género X": 2013
        }
    """

    # Verifica si el argumento es un str
    if not isinstance(genre, str):
        # Devuelve un mensaje de error
        return f"El género debe ser un texto de letras. Por favor, ingrese un género válido."

    # Convierte el género a título o a mayúsculas según corresponda
    if genre.upper() == "RPG":
        genre = genre.upper()
    else:
        genre = genre.title()

    # Verifica si el género existe
    if genre not in df_endpoints1_2.columns:
        # Devuelve un mensaje de error
        return f"Genre {genre} not found"

    # Filtra por género y resetea el índice para poder obtener el año de lanzamiento correcto
    df_genre = df_endpoints1_2[genre].reset_index()

    # Agrupa por año y calcula la suma total de horas jugadas para cada año
    playtime_by_year = df_genre.groupby('release_year')[genre].sum()

    # Encuentra el año con más horas jugadas
    max_playtime_year = (playtime_by_year.idxmax())
    response = {
        f"Año de lanzamiento con más horas jugadas para el género {genre}": max_playtime_year}

    return response

In [5]:
PlayTimeGenre('Casual')

{'Año de lanzamiento con más horas jugadas para el género Casual': 2015}

In [6]:
PlayTimeGenre('Action')

{'Año de lanzamiento con más horas jugadas para el género Action': 2012}

In [13]:
PlayTimeGenre('Casuals')

'Genre Casuals not found'

In [14]:
PlayTimeGenre(12)

'El género debe ser un texto de letras. Por favor, ingrese un género válido.'

### Endpoint 2


- def **UserForGenre( _`genero` : str_ )**:
  Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno:

```js
{
   "Usuario con más horas jugadas para Género X":"us213ndjss09sdf",
   "Horas jugadas":[
      {
         "Año":2013,
         "Horas":203
      },
      {
         "Año":2012,
         "Horas":100
      },
      {
         "Año":2011,
         "Horas":23
      }
   ]
}
```


In [16]:
def UserForGenre(genre: str):
    """
    Devuelve el usuario que acumula más horas jugadas para un género dado.
  
    Args:
      genre: El género del juego.
  
    Returns:
      El usuario con más horas jugadas y una lista de la acumulación de horas jugadas por año.
      Ejemplo de retorno:
          {
            "Usuario con más horas jugadas para Género X":"us213ndjss09sdf",
            "Horas jugadas":[
                {
                  "Año":2013,
                  "Horas":203
                },
                {
                  "Año":2012,
                  "Horas":100
                },
                {
                  "Año":2011,
                  "Horas":23
                }
            ]
          }
      """
    # Verificar si el argumento es un str
    if not isinstance(genre, str):
        # Devolver un mensaje de error
        return f"El argumento debe ser un texto de letras. Por favor, ingrese un género válido."

    # Convertir el argumento a título o a mayúsculas según corresponda
    if genre.upper() == "RPG":
        genre = genre.upper()
    else:
        genre = genre.title()

    if genre not in df_endpoints1_2.columns:
        # Devolver un mensaje de error
        return f"Genero {genre} no encontrado"

    df_genre = df_endpoints1_2[genre].reset_index()
    user_max_hours = df_genre.groupby('user_id')[genre].sum().idxmax()
    hours_by_year = df_genre[df_genre['user_id'] == user_max_hours].groupby('release_year')[genre].sum().reset_index()
    hours_by_year.columns = ['Año', 'Horas']
    hours_by_year = hours_by_year[hours_by_year['Horas'] > 0]
    hours_by_year['Horas'] = hours_by_year['Horas'].round().astype(int)

    response = {
        f"Usuario con más horas jugadas para Género {genre} ": user_max_hours,
        "Horas jugadas": hours_by_year.to_dict('records')
    }

    return response


In [17]:
UserForGenre('rpg')

{'Usuario con más horas jugadas para Género RPG ': 'shinomegami',
 'Horas jugadas': [{'Año': 1999, 'Horas': 6},
  {'Año': 2000, 'Horas': 140},
  {'Año': 2003, 'Horas': 8849},
  {'Año': 2004, 'Horas': 30},
  {'Año': 2005, 'Horas': 3},
  {'Año': 2006, 'Horas': 34},
  {'Año': 2007, 'Horas': 2278},
  {'Año': 2008, 'Horas': 22},
  {'Año': 2009, 'Horas': 23},
  {'Año': 2010, 'Horas': 690},
  {'Año': 2011, 'Horas': 351},
  {'Año': 2012, 'Horas': 459},
  {'Año': 2013, 'Horas': 2108},
  {'Año': 2014, 'Horas': 2456},
  {'Año': 2015, 'Horas': 2132},
  {'Año': 2016, 'Horas': 259}]}

In [18]:
UserForGenre('action')

{'Usuario con más horas jugadas para Género Action ': 'Sp3ctre',
 'Horas jugadas': [{'Año': 1995, 'Horas': 4},
  {'Año': 1997, 'Horas': 4},
  {'Año': 1999, 'Horas': 1},
  {'Año': 2000, 'Horas': 1177},
  {'Año': 2001, 'Horas': 4},
  {'Año': 2002, 'Horas': 4},
  {'Año': 2003, 'Horas': 129},
  {'Año': 2004, 'Horas': 2124},
  {'Año': 2005, 'Horas': 356},
  {'Año': 2006, 'Horas': 1504},
  {'Año': 2007, 'Horas': 1880},
  {'Año': 2008, 'Horas': 142},
  {'Año': 2009, 'Horas': 1813},
  {'Año': 2010, 'Horas': 1379},
  {'Año': 2011, 'Horas': 2591},
  {'Año': 2012, 'Horas': 6404},
  {'Año': 2013, 'Horas': 2036},
  {'Año': 2014, 'Horas': 2214},
  {'Año': 2015, 'Horas': 6572},
  {'Año': 2016, 'Horas': 503},
  {'Año': 2017, 'Horas': 722}]}

In [19]:
UserForGenre('Acitones')

'Genero Acitones no encontrado'

In [20]:
UserForGenre(123)

'El argumento debe ser un texto de letras. Por favor, ingrese un género válido.'

### Endpoint 3


- def **UsersRecommend( _`año` : int_ )**:
  Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

Ejemplo de retorno:

```js
[
  {
    "Puesto 1": "X",
  },
  {
    "Puesto 2": "Y",
  },
  {
    "Puesto 3": "Z",
  },
];
```


In [21]:
def UsersRecommend(year: int):
    """
    Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado.
    (reviews.recommend = True y comentarios positivos/neutrales)

    Args:
      year: El año a filtrar.

    Returns:
      El top 3 de juegos recomendados.
      Ejemplo de retorno:
          [
            {
                "Puesto 1":"X"
            },
            {
                "Puesto 2":"Y"
            },
            {
                "Puesto 3":"Z"
            }
          ]
    """
    # Comprueba si el año se encuentra en el DataFrame
    if ~df_endpoints3_4_5['posted_year'].isin([year]).any():
        print(f"Year {year} not found")
        return None

    # Filtra el DataFrame por año y recomendación
    df_year = df_endpoints3_4_5[(df_endpoints3_4_5['posted_year'] == year) & (
            df_endpoints3_4_5['recommend'] == True) & (df_endpoints3_4_5['sentiment_analysis'] > 1)]

    # Agrupa por juego y cuenta el número de recomendaciones para cada juego
    recommendations = df_year.groupby('item_name').size()

    # Ordena los juegos en función del número de recomendaciones y toma el top 3
    top_games = recommendations.sort_values(ascending=False).head(3)

    response = [{"Puesto {}".format(i + 1): game}
                for i, game in enumerate(top_games.index)]

    return response

In [22]:
UsersRecommend(2021)

Year 2021 not found


In [27]:
UsersRecommend(2015)

[{'Puesto 1': 'Counter-Strike: Global Offensive'},
 {'Puesto 2': "Garry's Mod"},
 {'Puesto 3': 'Unturned'}]

In [23]:
UsersRecommend(2010)

[{'Puesto 1': 'Team Fortress 2'},
 {'Puesto 2': 'Alien Swarm'},
 {'Puesto 3': "Garry's Mod"}]

### Endpoint 4


- def **UsersWorstDeveloper( _`año` : int_ )**:
  Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

Ejemplo de retorno:

```js
[
  {
    "Puesto 1": "X",
  },
  {
    "Puesto 2": "Y",
  },
  {
    "Puesto 3": "Z",
  },
];
```


In [28]:
def UsersWorstDeveloper(year: int):
    """
    Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

    Args:
      year: El año a filtrar.

    Returns:
      El top 3 de desarrolladoras con juegos MENOS recomendados
      Ejemplo de retorno:
          [
            {
                "Puesto 1":"X"
            },
            {
                "Puesto 2":"Y"
            },
            {
                "Puesto 3":"Z"
            }
          ]
    """
    # Comprueba si el año se encuentra en el DataFrame
    if ~df_endpoints3_4_5['posted_year'].isin([year]).any():
        print(f"Year {year} not found")
        return None

    # Filtra el DataFrame por año y no recomendación
    df_year = df_endpoints3_4_5[(df_endpoints3_4_5['posted_year'] == year) & (
            df_endpoints3_4_5['recommend'] == False) & (df_endpoints3_4_5['sentiment_analysis'] == 0)]

    # Agrupa por juego y cuenta el número de no recomendaciones para cada juego
    not_recommendations = df_year.groupby('developer').size()

    # Ordena los juegos en función del número de no recomendaciones y toma el top 3
    top_games = not_recommendations.sort_values(ascending=False).head(3)

    response = [{"Puesto {}".format(i + 1): game}
                for i, game in enumerate(top_games.index)]

    return response

In [29]:
UsersWorstDeveloper(2025)

Year 2025 not found


In [30]:
UsersWorstDeveloper(2015)

[{'Puesto 1': 'Bohemia Interactive'},
 {'Puesto 2': 'Facepunch Studios'},
 {'Puesto 3': 'Valve'}]

### Endpoint 5


def **sentiment_analysis( _`empresa desarrolladora` : str_ )**:
Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total
de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.

Ejemplo de retorno:

```js
{
   "Valve":[
      {
         "Negative":1352
      },
      {
         "Neutral":2202
      },
      {
         "Positive":4840
      }
   ]
}
```


In [31]:
def sentiment_analysis(developer: str):
    """
    Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.

    Args:
      developer: La desarrolladora a filtrar.

    Returns:
      El análisis de sentimiento para la desarrolladora dada.
      Ejemplo de retorno:
          {
            "Valve":[
                {
                  "Negative":1352
                },
                {
                  "Neutral":2202
                },
                {
                  "Positive":4840
                }
            ]
          }
    """
    # Convierte el nombre de la desarrolladora a minúsculas
    developer_lower = developer.lower()

    # Filtra el DataFrame por el nombre de la empresa desarrolladora
    df_dev = df_endpoints3_4_5[df_endpoints3_4_5["developer"].str.lower()
                               == developer_lower]

    # Verifica si el desarrollador ingresado existe en el DataFrame
    if df_dev.empty:
        print(f"Developer {developer} not found")
        return None

    # Contamos el número de registros por cada categoría de sentimiento
    sentiment_counts = df_dev['sentiment_analysis'].value_counts()
    negative_reviews = sentiment_counts.get(0, 0)
    neutral_reviews = sentiment_counts.get(1, 0)
    positive_reviews = sentiment_counts.get(2, 0)

    # Crear el diccionario con el nombre de la empresa desarrolladora como llave y la lista con los conteos como valor
    response = {
        developer_lower.capitalize(): [
            {"Negative": negative_reviews},
            {"Neutral": neutral_reviews},
            {"Positive": positive_reviews}
        ]
    }

    return response

In [37]:
sentiment_analysis('sega')

{'Sega': [{'Negative': 23}, {'Neutral': 9}, {'Positive': 59}]}

In [36]:
sentiment_analysis('sega')

{'Sega': [{'Negative': 23}, {'Neutral': 9}, {'Positive': 59}]}

In [35]:
sentiment_analysis('Ubisoft450')

Developer Ubisoft450 not found


## Recommendations

### Item-Item

+ def **game_recommender( *`item_name`* )**:
    Ingresando el nombre de un juego, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

In [39]:
def game_recommender(item_name: str):
    '''
    Muestra una lista de juegos similares a un juego dado.

    Args:
        item_name: El nombre del juego para el cual se desean encontrar juegos similares.

    Returns:
        Los 5 juegos recomendados.

    '''
    # Verificamos si el juego ingresado está en el dataframe
    if item_name not in df_similitud_items.index:
        return f"Game {item_name} not found"

    # Buscamos la fila del dataframe que corresponde al juego ingresado
    row = df_similitud_items.loc[item_name]

    # Ordenamos la fila de mayor a menor similitud
    row_sorted = row.sort_values(ascending=False)

    # Obtenemos los nombres de los 5 juegos más similares, excluyendo el mismo juego
    similar_games = row_sorted.index[1:6]

    # Creamos un diccionario vacío para guardar los juegos y sus índices
    recommendations = {}

    for i, juego in enumerate(similar_games, start=1):
        # Asignamos el índice y el nombre del juego al diccionario
        recommendations[i] = juego
    return recommendations

In [40]:
df_similitud_items.columns

Index(['100% Orange Juice', '8BitMMO', 'A Bird Story',
       'A Story About My Uncle', 'APB Reloaded', 'ARK: Survival Evolved',
       'ARMA: Cold War Assault', 'Ace of Spades: Battle Builder',
       'AdVenture Capitalist', 'Age of Empires II HD',
       ...
       'Worms Revolution', 'XCOM: Enemy Unknown', 'XCOM® 2',
       'Yet Another Zombie Defense', 'You Have to Win the Game',
       'Zombie Army Trilogy', 'how do you Do It?', 'the static speaks my name',
       'theHunter Classic', 'theHunter: Primal'],
      dtype='object', name='item_name', length=548)

In [41]:
game_recommender('Bully: Scholarship Edition')

{1: 'APB Reloaded',
 2: 'XCOM® 2',
 3: 'Sonic & All-Stars Racing Transformed',
 4: 'Darkest Dungeon®',
 5: 'Tabletop Simulator'}

### User-Item

+ def **recomendacion_usuario( *`user_id`* )**:
    Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

In [17]:
def user_recommendation(user_id: str):
    """
    Esta función recibe el id de un usuario y retorna una lista con 5 juegos recomendados para dicho usuario, basándose en la similitud de los items y los ratings de otros usuarios.

    Args:
        user_id: str El id del usuario para el que se quiere obtener las recomendaciones.

    Return:
        Una lista con los nombres de los 5 juegos con mayor rating predicho para el usuario.
    """

    # Ignoramos las operaciones de división inválidas
    np.seterr(divide='ignore', invalid='ignore')
    # Verificamos si el usuario ingresado está en el dataframe de ratings
    if user_id in df_matrix.index:
        # Obtenemos la fila del dataframe de ratings que corresponde al usuario ingresado
        fila_usuario = df_matrix.loc[user_id]
        # Creamos una lista vacía para guardar los juegos que el usuario no ha jugado
        juegos_no_jugados = []
        # Recorremos la fila del usuario con un bucle for
        for juego, rating in fila_usuario.items():
            # Si el rating es cero, significa que el usuario no ha jugado ese juego
            if rating == 0:
                # Añadimos el juego a la lista de juegos no jugados
                juegos_no_jugados.append(juego)
        # Creamos una lista vacía para guardar las predicciones de ratings para los juegos no jugados
        predicciones = []
        # Recorremos la lista de juegos no jugados con otro bucle for
        for juego in juegos_no_jugados:
            # Obtenemos la columna del dataframe de similitud de items que corresponde al juego
            columna_juego = df_similitud_items.loc[juego]
            # Calculamos el producto punto entre la fila del usuario y la columna del juego
            producto = np.dot(fila_usuario, columna_juego)
            # Calculamos la suma de las similitudes de los juegos que el usuario ha jugado
            suma_sim = np.sum(columna_juego[fila_usuario != 0])
            # Calculamos la predicción de rating como el cociente entre el producto y la suma
            prediccion = producto / suma_sim
            # Añadimos una tupla con el juego y la predicción a la lista de predicciones
            predicciones.append((juego, prediccion))
        # Ordenamos la lista de predicciones de mayor a menor rating
        predicciones_ordenadas = sorted(
            predicciones, key=lambda x: x[1], reverse=True)
        diccionario = {}
        # Recorremos las predicciones ordenadas con un bucle for
        for i, tupla in enumerate(predicciones_ordenadas[:5], start=1):
            # Asignamos el índice, el nombre del juego y el rating predicho al diccionario
            diccionario[i] = (tupla[0])
        # Retornamos el diccionario de juegos recomendados
        return diccionario
        # Creamos un diccionario vacío para guardar los juegos y sus ratings predichos

    else:
        # Retornamos un mensaje de error o una lista vacía
        return "El usuario ingresado no se encuentra en la fuente de datos"

In [19]:
user_recommendation("yoshipowerz")

{1: 'A Bird Story',
 2: 'A Story About My Uncle',
 3: 'ARMA: Cold War Assault',
 4: 'Brutal Legend',
 5: 'AdVenture Capitalist'}

In [6]:
df_matrix.index

Index(['-SEVEN-', '091263', '1011001', '12345678901234567890123456567890',
       '1234567io9872345678765432', '12779', '131312', '1337lolroflmao',
       '1626466724893520', '17101710',
       ...
       'yoshipowerz', 'yotuic', 'you_re_ded', 'youngbenaffleck', 'zaaikbr',
       'zachwgtv', 'zakbot', 'zaukster', 'zayyntt', 'zyr0n1c'],
      dtype='object', name='user_id', length=1391)